# DC Tools

In [1]:
import pandas as pd
import requests
import numpy as np
import re
import json
from pathlib import Path
from io import StringIO

In [2]:
data = requests.get('https://raw.githubusercontent.com/designunit/dc-data/master/data.csv').text
df = pd.read_csv(StringIO(data), index_col='alias')

## DC12 Setup

In [3]:
usecols = ['code', 'name', 'description', 'units', 'pic link']
df12 = pd.read_excel('DC CATALOGUE 12.xlsx', usecols=usecols)
df12.columns = ['alias', 'title', 'description', 'units', 'image_file']
df12.set_index('alias', inplace=True)

# delete duplicates
df12 = df12.loc[~df12.index.duplicated(keep='first')]

In [4]:
def get_units(value):
    if value == 'running meter':
        return 'running_meter'
    if value == 'pcs':
        return 'piece'
    return value
df12['units'] = df12['units'].map(get_units)

In [5]:
def cap(x):
    if isinstance(x, str):
        return str(x).capitalize().strip()
    return x

df12['title'] = df12['title'].map(cap)
df12['description'] = df12['description'].map(cap)

In [6]:
def file_name(x):
    if isinstance(x, str):
        x = x.replace('\\', '/')
        return Path(x).name
    return x
df12['image_file'] = df12['image_file'].map(file_name)

In [7]:
df12['code'] = df['code']

## Setup merged DF

In [8]:
df = df12[df12['code'].notnull()]
df = df.reindex(columns=['code', 'name', 'units', 'image_file', 'description'])
df.head(10)

,code,name,units,image_file,description
alias,,,,,
P_C_01_a,PAV-PRD1-MAT1-2-PAT1-COL2-BST1,NaN,m2,P_C_01.jpg,"Одноцветное мощение, светло-серая плитка, с гл..."
P_C_01_b,PAV-PRD1-MAT1-2-PAT1-COL2-BST2,NaN,m2,P_C_01.jpg,"Одноцветное мощение, светло-серая плитка, с гл..."
P_C_02_a,PAV-PRD1-MAT1-5-PAT1-COL2-BST1,NaN,m2,P_C_02.jpg,"Одноцветное мощение, светло-серая плитка, с гл..."
P_C_02_b,PAV-PRD1-MAT1-5-PAT1-COL2-BST2,NaN,m2,P_C_02.jpg,"Одноцветное мощение, светло-серая плитка, с гл..."
P_C_03_a,PAV-PRD1-MAT1-4-PAT1-COL1-BST1,NaN,m2,P_C_03.jpg,"Одноцветное мощение, черная плитка с гладкой и..."
P_C_03_b,PAV-PRD1-MAT1-4-PAT1-COL1-BST2,NaN,m2,P_C_03.jpg,"Одноцветное мощение, черная плитка с гладкой и..."
P_C_04_a,PAV-PRD1-MAT1-5-PAT1-COL2-BST1,NaN,m2,P_C_04.jpg,"Одноцветное мощение, черная плитка с гладкой п..."
P_C_04_b,PAV-PRD1-MAT1-5-PAT1-COL2-BST2,NaN,m2,P_C_04.jpg,"Одноцветное мощение, черная плитка с гладкой п..."
P_C_05_a,PAV-PRD1-MAT1-PAT2-COL2-BST1,NaN,m2,P_C_05.jpg,"Одноцветное мощение, светло-серая плитка с гла..."


## Export

In [9]:
df.to_csv('data.csv', sep=',')